## LNC2

If a document is replicated k-times, its retrieval status value should not be lower than that of its un-replicated variant. 
To generate the dataset, generate documents that are k-times copied over and over again.                        

In [58]:
#How many documents has padding on them?
import os
from tqdm.autonotebook import tqdm
data_dir = "/ssd2/arthur/TREC2019/data"
dev_file = os.path.join(data_dir, "dev-triples.top100")
shorter_documents_lens = []
with open(dev_file) as inf:
    for line in tqdm(inf, total=519300):
        doc = eval(line.split("\t")[1])
        if len(doc)<512:
            shorter_documents_lens.append(len(doc))

KeyboardInterrupt: 

In [32]:
len(shorter_documents_lens)/519300

0.05712690159830541

## TFC-1
Given two documents, If a query term occours more in one than in the other, the first score should be higher than the second.

In [ ]:
!ls /ssd2/arthur/

In [41]:
#load query-doc pairs scores
run_file = "/ssd2/arthur/TREC2019/data/runs/bert_eval/distilBert-0.85.res"
scores = {}

with open(run_file) as inf:
    for line in inf:
        topic_id, _, doc_id, _, score, _ = line.split()
        pair_id = "{}-{}".format(topic_id, doc_id)
        scores[pair_id] = float(score)

In [110]:
satisfies

2370

In [112]:
last_topic_id

'320792'

In [121]:
#for each query, generate 99*100 pairs. 
delta = 3
last_topic_id = -1
docs = {}

all_candidates = 0
satisfies = 0
with open(dev_file) as inf:
    for index, line in tqdm(enumerate(inf), total=519300):
        if index+1 % 10000 == 0:
            print(satisfies/all_candidates)
        topic_id, doc_id = line.split("\t")[0].split("-")
        if topic_id != last_topic_id and last_topic_id !=-1:
            topic_candidates, topic_satisfies = check_TFC1_single_topic(docs, query, last_topic_id)
            all_candidates += topic_candidates
            satisfies += topic_satisfies
            docs = {}
        last_topic_id = topic_id
        doc = eval(line.split("\t")[1])
        sep_id = doc.index("[SEP]")
        document = doc[sep_id+1:-1]
        docs[doc_id] = document
        query = doc[1:sep_id]

KeyboardInterrupt: 

In [113]:
from collections import Counter

def check_TFC1_single_topic(docs, query, last_topic_id):
    number_of_candidates = 0
    query_terms = set(query)
    satisfies = 0
    for di_id in docs.keys():
        di = docs[di_id]
        di_terms = set(di)
        di_terms_counter = Counter(di)
        di_pair_id = "{}-{}".format(last_topic_id, di_id)
        sum_occurences_di = sum([di_terms_counter[term] for term in query_terms])
        for dj_id in docs.keys():
            if di_id == dj_id:
                continue
            dj = docs[dj_id]
            if abs(len(di)-len(dj)) > delta:
                continue
            dj_terms = set(dj)
            dj_terms_counter = Counter(dj)
            dj_pair_id = "{}-{}".format(last_topic_id, dj_id)
            sum_occurences_dj = sum([dj_terms_counter[term] for term in query_terms])
            # does not satisfies axiom premisse
            if sum_occurences_di <= sum_occurences_dj:
                continue
            number_of_candidates+=1
            # Satisfies. Check if score is also bigger.
            if scores[di_pair_id] > scores[dj_pair_id]:
                satisfies += 1
    return number_of_candidates, satisfies
check_TFC1_single_topic(docs, query, last_topic_id)

KeyError: '320792-D3126539'

In [59]:
print(doc[1:sep_id])

['what', 'is', 'det', '##ox', 'treatment', '[SEP]', 'd', '##8', '##16', '##27', '##7']


## TFC2

In [8]:
import os
from tqdm.auto import tqdm
data_home = "/ssd2/arthur/TREC2019/data/"
datasets = ["fulldev", "dev", "test"]
methods = {
    "QL": os.path.join(data_home, 'runs', "{}_distilBert-0.0.run"),
    "BERT": os.path.join(data_home, 'runs', "{}_distilBert-1.0.run"),
    "QL+BERT": os.path.join(data_home, 'runs', "{}_distilBert-0.85.run")
}
for method in methods.keys():
    for dataset in datasets:
        last_topic_id  = -1
        docs = {}
        scores = {}
        
        all_candidates = 0
        satisifies = 0
        query = None
        run_file = methods[method].format(dataset)
        for line in open(run_file):
            topic_id, _, doc_id, _, score, _ = line.split()
            pair_id = "{}-{}".format(topic_id, doc_id)
            scores[pair_id] = float(score)
        triples_file = os.path.join(data_home, 'triples-tokenized', '{}-triples.top100'.format(dataset))
        for line in tqdm(open(triples_file), desc="reading file {}".format(triples_file)):
            topic_id, doc_id = line.split("\t")[0].split("-")
            if topic_id != last_topic_id and last_topic_id != -1:
                break
            last_topic_id = topic_id
            doc = eval(line.split("\t")[1])
            sep_id = doc.index("[SEP]")
            document = doc[sep_id+1:-1]
            docs[doc_id] = document
            query = doc[1:sep_id]
        break
    break
            
            


In [11]:
last_topic_id

'174249'

In [ ]:
TF

In [28]:
query_terms.intersection(TF_d3.keys())

{'does', 'in', 'money', 'to'}

In [62]:
from itertools import product
from collections import Counter
# def TFC2(docs, query, last_topic_id, scores):
    #this is a three-way document comparison.
number_of_candidates = 0
query_terms = set(query)
satisfies = 0
delta = 4
for d1_id, d2_id, d3_id in tqdm(product(docs.keys(), docs.keys(), docs.keys())):
    if d1_id == d2_id or d2_id == d3_id or d1_id == d3_id:
        continue
    d1 = docs[d1_id]
    TF_d1 = Counter(d1)

    d2 = docs[d2_id]
    TF_d2 = Counter(d2)
    
    d3 = docs[d3_id]
    TF_d3 = Counter(d3)
    # Every document must contain at least one query term
    if len(query_terms.intersection(TF_d1)) < 1 or len(query_terms.intersection(TF_d2)) < 1 or len(query_terms.intersection(TF_d3)) < 1:
        continue
    # Doc lenghts should be within an acceptable range
    if abs(len(d3) - len(d1)) > delta or abs(len(d2) - len(d1)) > delta or abs(len(d3) - len(d2)) > delta:
        continue
    
    sum_TF_d1 = sum([TF_d1[w] for w in query_terms])
    sum_TF_d2 = sum([TF_d2[w] for w in query_terms])
    sum_TF_d3 = sum([TF_d3[w] for w in query_terms])
    
    if sum_TF_d3 <= sum_TF_d2 or sum_TF_d2 <= sum_TF_d1:
        continue
    for w in query_terms:
        if (TF_d2[w] - TF_d1[w]) == (TF_d3[w] - TF_d2[w]):
            continue
    number_of_candidates += 1
    d1_pair_id = "{}-{}".format(last_topic_id, d1_id)
    d2_pair_id = "{}-{}".format(last_topic_id, d2_id)
    d3_pair_id = "{}-{}".format(last_topic_id, d3_id)
    if (scores[d2_pair_id]-scores[d1_pair_id]) > (scores[d3_pair_id] - scores[d2_pair_id]):
        satisfies+=1
        break
    

## M-TDC 
First, we need to compute IDFs on the corpus.

In [64]:
import pickle
IDFs = pickle.load(open("/ssd2/arthur/TREC2019/data/docs/IDFS/IDFS-FULL", 'rb'))

collections.Counter